# Chapter 2: Learning Python and solving algebraic equations
Introduction to the Phys212/Biol212 class
### by Ilya Nemenman, 2016-2020

Make sure you read Chapters 1 and 2 of the *Student Guide* before this notebook, and that you read Chapters 3 and 4 in parallel with this notebook. Make sure also that you in detail study the code for the cells that I provide below, not just the output of the cells. This is essential if you want to pick up on python programming quickly.

## Some Python practice 
Let's first pre-emptively review a few elements in Python coding that many of you probably will have problems with otherwise. 

### Assignments and comparisons
Let's execute the following code:

In [1]:
a = 0
a == 1
print (a)
a = 1
print(a)
b = (a == 1)
print(b)

0
1
True


Note that the first line above assigns the value of 0 to the variable `a`. The second line does not do the assignment, but instead compares `a` to 1. Since the result of the comparison is not stored anywhere, it disappears. We can verify that `a` was not changed by the comparison by simply printing it. The next line, in contrast, assigns 1 to `a`. We verify that this, in fact, happened by printing the variable. We again do the comparison of `a` to 1. Notice, however, that this time we store the result of this comparison in a variable $b$, which has the value of `True`, as we can see by printing it.

### Vector operations
You have created numpy arrays in your first lab. Why do we bother with this concept? Arrays allow us to perform vector operations -- instead of, for example, adding two arrays element by element, or copying an array element by element we can do either of these tasks in just one single operation. Let's see how this works.

In [11]:
import numpy as np #importing the numpy module

L = 10  # the size of the arrays we will create
a = np.arange(L)
b = np.ones(L)
print(a)
print(b)

c = a + b
print(c)

[0 1 2 3 4 5 6 7 8 9]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


2.0

Notice how we didn't have to create `c` separately, and in a single addition operation we were able to add all elements of `a` and `b` into `c`. Such vector operations are, of course, more elegant than doing the same element by element. Importantly, they are also faster, as we will verify shortly. 

Note that the array `a` is an array of integers. While the arrays `b` and `c` are arrays of real (rather, floating point -- we will discuss what this means in a few lectures) numbers. Real and integer numbers are very different types of objects, and are represented very differently in a computer memory, even if sound very similar to us. Luckily, Python is very forgiving about conversion of types of variables between various integer and real types (there are many of either kind). You can add integers to reals, for example, as the line `c = a+b` above illustrates. For this, Python will automatically convert the array of integers into reals, and then add the real numbers. There are a few cases, however, where one has to be careful, and you surely will run into them. Let's try

In [13]:
print(c[1])
print(c[1.0])

2.0


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

Note that the first line works, while the second produces an error. You can only refer to an integer position in an array. There's no element with a number 1.5, 2.7, or 1.0. 

Now let's verify that vector operations are much faster than element by element ones. To measure how long something takes, we need to load a new module `Time`.

In [3]:
import time
dir(time)

['_STRUCT_TM_ITEMS',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'altzone',
 'asctime',
 'clock',
 'ctime',
 'daylight',
 'get_clock_info',
 'gmtime',
 'localtime',
 'mktime',
 'monotonic',
 'monotonic_ns',
 'perf_counter',
 'perf_counter_ns',
 'process_time',
 'process_time_ns',
 'sleep',
 'strftime',
 'strptime',
 'struct_time',
 'time',
 'time_ns',
 'timezone',
 'tzname',
 'tzset']

>### Your turn 2.1
Explore what different functions in this module do.

We will be using the function `time.time()`, which return the time in seconds since the beginning of some standardized epoch as a floating point number. Notice that the function `time.time()` must be called with the parenthesis at the end. Indeed, let's try:

In [4]:
print(time.time)
print(time.time())

<built-in function time>
1579790522.700278


>### Your turn 2.2
Explain the results of the Python output above.

Now let's see how long vector and non-vector versions of the same operations take. 

In [5]:
L = 100000        # length of the vectors we will operate with
a = np.arange(L)  # creating these long vectors
b = np.ones(L)

# now let's add a and b as a vector  operation and see how long it takes
t = time.time()   # store the current time in the variable t
c = a+b
print ('The vector addition took', time.time()-t, 'sec.')

The vector addition took 0.0016257762908935547 sec.


>### Your turn 2.3
Run the cell above multiple times. Does it always time the same duration of time to execute the same command?

Now let's do the same operation using element by element addition.

In [6]:
t = time.time()  # store the current time in the variable t

c = np.zeros(L)  # preallocate memory for an empty array
for i in np.arange(L):  # do element-by-element addition
    c[i] = a[i] + b[i]
    
print ('Element by element addition with pre-allocated memory took', time.time() - t, 'sec.')

Element by element addition with pre-allocated memory took 0.3539109230041504 sec.


Notice how much longer it took to execute the same operation. But this is not the worst. Let's try the following.

In [7]:
t = time.time()  # store the current time in the variable t

c = np.arange(0) # create a null array (array with no elements)
print(c)

for i in np.arange(L): # do element-by-element addition while growing the array
    c = np.hstack((c, a[i] + b[i]))
    
print ('Element by element addition with pre-allocated memory took', time.time() - t, 'sec.')

[]
Element by element addition with pre-allocated memory took 3.1772260665893555 sec.


>### Your turn 2.4
Why do we use double perentheses ((...)) in the call to `np.hstack()`?

Why does this happen? The vector operations are the fastest -- they are encoded using low-level computer language, and are optimized internally. The element-wise edition takes longer because at every point in the cycle the Python interpreter has to do a variety of checks, such as is `a` an array? Does it have an element `i`? And the same for `b` and `c`, and all of this takes time. Finally, in the third version, every time we stack the arrays together, a new array gets created. **We will discuss this in class in more detail, and I wil add a figure later to explain this.**

## Solving a quadratic equation

Let's say we want to solve a quadratic equation $$ax^2+bx+c=0.$$ The two solutions of this are given by $$x_{1,2}=\frac{-b\pm \sqrt{b^2-4ac}}{2a},$$
where $D=b^2-4ac$ is called the discriminant. Let's write a short script to solve this. 

We start with the initialization block. Normally we would import `numpy` and other modules here as well, but now we do not need to do this, this numpy was just imported above.

In [8]:
a = 1.0  #initializing parameter values
b = 2.0
c = -2.0

Now we calculate the square root of the determinant using the `numpy.np` function, and print the two roots.

In [9]:
sD = np.sqrt(b**2-4*a*c)
x1 = (-b+sD)/(2*a)
x2 = (-b-sD)/(2*a)

print(x1)
print(x2)

0.7320508075688772
-2.732050807568877


This code would work well often, but not always. For example, change `c` to 2.0 above and rerun the cell. What happened? This introduces the idea of checking of exceptions: what if the discriminant is zero? or negative? or if `a` is equal to 0? 

>### Your turn 2.5
Change the code for solving the quadratic equations to address all possible exceptions you can think of, so that the code runs for all values of the parameters `a`, `b`, and `c`. 

>### Track 2
Rewrite the code above to work with complex numbers.

>### Your turn 2.6
Additional exercise: Evaluate a binary log (log base 2) of a number 42 using at least three different sets of commands imported from `numpy`. Verify your result by taking 2 to the appropriate power and seeing if you get 42 back. For this, you will need to remind yourself how logarithms with different bases relate to each other.

## The first model: Solving for an equilibrium position of charges, a light version
We will now turn to our first model, and will follow the steps involved in constructing it, solving it on a computer, and verifying the solution

### The problem formulation
A charged particle free to move on a rod is put in the between two particles with an electric charge of the same sign affixed to the end of the rod. Where will the charge come to rest?

### Analysis
Analysis is always the first step in solving the problem. We start with asking hwat kind of model needs to be built for this problem. The model is definitely a continuous space model -- position of the particle is a continuous variable. The model is also deterministic -- there's not a stochastic element present. However, the next question -- whether the model is dynamic or static -- is already harder. At the first glance, this is a dynamic model, since we are talking about *motion* of a charge. On the other hand, we are being asked only about the final equilibrium position. Then maybe this is a static model after all -- we only need to know where the charge comes to rest. IN fact, one can solve the model both ways: either by writing down the equations of motion, modeling the motion, and then seeing where the charges settle, or by finding the equlibrium position directly, which is what we will do here.

In any case, whether we decide to solve this as a dynamic or as a static model, to ensure that there is a resting point for the charge, there has to be friction, so that the particle does not keep oscillating infinitely. Nothing is mentioned about this in the problem formulation -- but it seems like this is presumed. If we were to write a report about this problem, we would definitely need to mention this consideration  as a part of our discussion of the problem setup. Is friction the only thing we need? Is it guaranteed that with friction the particle will come to a rest? Same sign particles repel. And the repulsion is stronger when they come closer. So a particle between two other fixed one will be pushed away from both ends -- it seems reasonable that it will come to an equilibrium point at some time if friction robs it of energy. So the solution will exist -- it now makes sense to proceed to finding it by building the model.

### Model building
The equilibrium for the system will be given by $\sum \vec{F}=0$, so that if a particle has zero velocity at this point (which the friction will achieve), it stays there. Since the particle is moving on a rod -- in one dimension -- this is equivalent to $\sum {F}=0$. There are two Coulomb forces acting on the particle from the two end charges. Let's say the charges are at positions $p_1$ and $p_2$, and have charges $q_1$ and $q_2$ respectively. Let's suppose $p_1<x<p_2$, where $x$ is the position of the moving charge. The forces acting on the moving charge, which we call $q$ are then $\sum F= \frac{qq_1}{(x-p_1)^2}-\frac{qq_2}{(x-p_2)^2}$. Thus to solve the problem, we need to solve the following equation for $x$: $\frac{qq_1}{(x-p_1)^2}-\frac{qq_2}{(x-p_2)^2}=0$, or, cancelling $q$, $\frac{q_1}{(x-p_1)^2}-\frac{q_2}{(x-p_2)^2}=0$. Interestingly, the only variables we need for initialization are the three charge values, and the two end charge positions -- the initial position of the moving charge, and charge value itself seem to be not needed. Multiplying the equation by the two denominators (which, luckily, cannot be zero), we get: ${q_1}{(x-p_2)^2}-{q_2}{(x-p_1)^2}=0$. Simplifying the equation, we get: $$(q_1-q_2) x^2 +2(-q_1p_2+q_2p_1)x+(q_1p_2^2-q_2p_1^2)=0 $$. So it seems that solving this problem is equivalent to solving a simple quadratic equation.

>### Your turn 2.7
Explore the markdown code of the text above. Notice how we write equations using $\LaTeX$. Try to pick up some $\LaTeX$ in the course of this semester. This will be useful for your future carrers as scientists. Minimally, figure out how we use `$` signs for encapsulating equations, `$$` for haveing equations in their own display lines, symbols `^` and `_` for superscripts and subscripts, and a command `\frac{}{}` for writing fractions. To verify your understanding, write a formula for a ratio of two quadratic polynomials in $x$.

### Model implementation
As suggested earlier, to build a model, we start with large blocks describing chunks of the solution of the problem. We progressively break them apart into smaller and smaller blocks, until we are able to write the code for each one of the blocks. At the same time, as we are implementing the blocks, we collect pieces that need to be initialized in the initialization block, and need to be reported in the termination block. Here we realize that, in the previous lecture, we wrote a simple program that solves the quadratic equation $ax^2+bx+c=0$. Thus implementing the model is equivalent to taking the previously written solution for the quadratic equation. Thus our implementation will consist of just three blocks: (1) initialization, where we assign  $a=q_1-q_2$, $b=2(-q_1p_2+p_2x_1)$, and $q_1p_2^2+q2p_1^2$; (2) solving the quadratic equation and getting its two solutions; and (3) termination, where, of the two solutions, we then output the one that falls between $p_1$ and $p_2$. 

### Model verification
To verify that the solution is correct, we should try it in cases where the solution is easy to guess or to estimate otherwise, and compare. For example, we can try $q_1=q_2$, where it's clear that the solution should be $x=(p_1+p_2)/2$. We could also choose a few conditions with different $q_1$, and we would expect that growing $q_1$ should push the equilibrium farther from the left end.

### Discussion
Having solved the problem, we would usually discuss our findings here, and reiterate various constraints and assumptions that entered the solution. However, this problem is too simple too write much here.

>### Your turn 2.8
Write a model for solving for the equilibrium position of the charge.

>### Your turn 2.9
Verify that the charge settles exactly in the middle between the two end charges when the latter are equal. Think of two more substantially different verification steps and implement them.